In [1]:
import gym
import json
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from gym import wrappers




In [2]:
from env.StockTradingEnv import StockTradingEnv
from networks.DQNac import MyModel
from networks.DQNac import StockActor, StockCritic, DQN

In [3]:
df = pd.read_csv('./data/AAPL.csv')
df = df.sort_values('Date')

# The algorithms require a vectorized environment to run
env = StockTradingEnv(df)

base_action = np.array([0,0.5])

obs = env.reset()
for i in range(5):
    action, _states = (base_action, None)
    obs, rewards, done, info = env.step(action)
#print(obs)
obs = env.reset()
obs=np.expand_dims(obs,axis=0)
obs=np.expand_dims(obs,axis=0)
print(obs)

[[[[1.92080002e-02 1.92300004e-02 1.94660004e-02 1.95800004e-02
    1.96979996e-02 1.98820008e-02]
   [1.93759994e-02 1.94479996e-02 1.95140000e-02 1.96380004e-02
    1.98740006e-02 2.01360000e-02]
   [1.91220002e-02 1.92080002e-02 1.93600006e-02 1.93720002e-02
    1.95960006e-02 1.98640000e-02]
   [1.91940002e-02 1.94479996e-02 1.95000000e-02 1.95960006e-02
    1.98320008e-02 2.01059998e-02]
   [1.57370232e-02 1.48620457e-02 1.30925328e-02 2.27945857e-02
    2.21524388e-02 3.23164277e-02]
   [4.65661288e-06 4.65661288e-06 0.00000000e+00 0.00000000e+00
    0.00000000e+00 0.00000000e+00]]]]


mod = MyModel(32, (1,4), (1,1))

print(obs)
z = mod.predict(obs)
print(z)


act = StockActor()
act_target = StockActor()
output_action = act(obs)
output_action_tar = act_target(obs)
print("output_action")
print(output_action)
print(" ")
print("output_action_tar")
print(output_action_tar)
print(" ")
output_action = act(obs)
output_action_tar = act_target(obs)
print("output_action2")
print(tf.squeeze(output_action))
print(" ")
print("output_action_tar2")
print(output_action_tar)
print(" ")

#act.model.summary()
#print(act.model.layers[3].get_weights())
#act.model.layers[3].set_weights([np.array([[-0.3]], dtype=np.float32), np.array([0.1], dtype=np.float32)])
#print(act.model.layers[3].get_weights())

crit = StockCritic()
act = StockActor()

output_action = act(obs)
print(output_action)
output_action = tf.squeeze(output_action)
output_crit = crit([obs,output_action])
print(output_crit)

    
gamma = 0.99
max_experiences = 1000
min_experiences = 365
batch_size = 32
lr = 1e-2
D = DQN( gamma, max_experiences, min_experiences, batch_size, lr)
observations = env.reset()
observations = np.expand_dims(observations,axis=0)
observations = np.expand_dims(observations,axis=0)
D.act(observations)

a = act.model.trainable_variables
print(a)
print(" ")
print(" ")
print(" hjkbedfhugqdfhiqdfbhiqdf")
print(" ")
print(" ")
b = act.trainable_variables
print(b)


#model = MyModel(32, (1,4), (1,1))
#output=tf.squeeze(model(obs))

crit = StockCritic()
crit_tar = StockCritic()
crit.action = tf.squeeze(output_action)
crit_tar.action = tf.squeeze(output_action)
output = crit(obs)
output_tar = crit_tar(obs)
print("output")
print(output)
print(" ")
print("output_tar")
print(output_tar)
print(" ")
crit.action = tf.squeeze(output_action_tar)
crit_tar.action = tf.squeeze(output_action)
output = crit(obs)
output_tar = crit_tar(obs)
print("output2")
print(output)
print(" ")
print("output_tar2")
print(output_tar)
print(" ")
print(crit.trainable_variables)

### Additional functions

In [4]:

def fill_buffer(env, DQN):
    
    rewards = 0
    iter = 0
    done = False
    observations = env.reset()
    observations = np.expand_dims(observations,axis=0)
    observations = np.expand_dims(observations,axis=0)
    steps = 0
    it=0
    while it<DQN.min_experiences:
        
        #print(it)
        #observations = env.reset()

        action = DQN.act(observations) # observations is actually a single "state" ie past 5 days
        action = np.tanh(tf.squeeze(action)+np.random.normal(0,0.05)) #avoiding small action values in the begining ->[2 0] to frequently
        #print(action)
        action = DQN.convert_action(action)
         
        #print(action)
        prev_observations = observations
        observations, reward, done, _ = env.step(action)
        #print(reward)
        observations = np.expand_dims(observations,axis=0)
        observations = np.expand_dims(observations,axis=0)
        rewards += reward    # sum of gain_net_worth

        if done :
            print("DONE")
#             reward = -200
            env.reset()
        steps += 1
        if steps >= 300: # Limiting the number of steps
            print("STEPS")
            observations = env.reset()
            observations = np.expand_dims(observations,axis=0)
            observations = np.expand_dims(observations,axis=0)
            steps = 0
        
        obs = tf.squeeze(observations)
        obs = np.expand_dims(obs,axis=0)
        exp = {'s': prev_observations, 'a': action, 'r': reward, 's2': obs, 'done': done}
        DQN.add_experience(exp)
        
        it += 1
    return " Done "

def DDPG():
    
    gamma = 0.99
    max_experiences = 1000
    min_experiences = 365
    batch_size = 32
    lr = 1e-2
    
    M = 1
    T = 1
 
    D = DQN( gamma, max_experiences, min_experiences, batch_size, lr)
    a = fill_buffer(env, D)
    #print(a)
    
    for i in range(M):
        D.train(env,T)
            

In [5]:
DDPG()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

STEPS

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you in